In [ ]:
from pathlib import Path
from dotenv import load_dotenv
import matplotlib.pyplot as plt

import warnings
import numpy as np
import pandas as pd
import hopsworks
import datetime
import os
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score

import openmeteo_requests

import requests_cache
from retry_requests import retry

warnings.filterwarnings("ignore")


In [ ]:
load_dotenv("../.env", override=True)
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
if HOPSWORKS_API_KEY is None:
    raise RuntimeError(
        "HOPSWORKS_API_KEY not found. "
    )


project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project="project_scalable")
fs = project.get_feature_store()
today = datetime.date.today()

In [ ]:
mr = project.get_model_registry()
model = mr.get_model(
    name="energy_predict", 
    version = 1,
)

saved_model_dir = model.download()

retrieved_xgboost_model = XGBRegressor()
retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")
retrieved_xgboost_model



In [ ]:
feature_view = fs.get_feature_view(
    name="weather_only_forecast",
    version=1,
)

data = feature_view.get_batch_data(
    start_time=today
)

data


In [ ]:

features = [
    "temperature_2m",
    "shortwave_radiation",
    "cloud_cover",
    "wind_speed_10m",
    "precipitation",]

X_forecast = data[features]

data["predicted_mwh"] = retrieved_xgboost_model.predict(X_forecast)
data["predicted_mwh"] = np.maximum(data["predicted_mwh"], 0)

print(data[["datetime", "predicted_mwh"]].head(10))

In [ ]:
data["datetime"] = pd.to_datetime(data["datetime"])
daily_forecast = (
    data
    .set_index("datetime")["predicted_mwh"]
    .resample("D")
    .sum()
    .reset_index()
)

print(daily_forecast.head())

In [ ]:
monitor = fs.get_or_create_feature_group(
    name='energy_prediction_fg',
    version=1,
    primary_key=[ 'datetime'],
    description='Predicted energy',
    event_time='datetime',
)

monitor.insert(data, wait = True)

monitoring_df = monitor.read()
monitoring_df

In [ ]:
import matplotlib.pyplot as plt

plots_dir = os.path.join(os.path.dirname(os.getcwd()), "plots")
os.makedirs(plots_dir, exist_ok=True)

plt.figure(figsize=(10, 4))
plt.plot(daily_forecast['datetime'], daily_forecast['predicted_mwh'], marker='o')
plt.xlabel("Date")
plt.ylabel("Predicted MWh")
plt.title("Weekly Solar Energy Prediction (Daily)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("../plots/daily_forecast.png", dpi=150)
plt.show()


In [ ]:
# Filter for today's date
today = pd.to_datetime("today").date()
df_today = data[data['datetime'].dt.date == today]

plt.figure(figsize=(12, 4))
plt.plot(df_today['datetime'], df_today['predicted_mwh'], marker='o')
plt.xlabel("Hour")
plt.ylabel("Predicted MWh")
plt.title(f"Solar Energy Prediction for {today}")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("../plots/hourly_forecast.png", dpi=150)
plt.show()

print('done')